In [1]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import os
import glob

## Get Link

In [3]:
url = 'https://www.innovestx.co.th/faqs'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}
manual_links = set()
for i in range(1,5):
    link = f'https://www.innovestx.co.th/products/product-user-guide/detail/{i}'
    response = requests.get(link,headers=headers)
    page_html = response.content
    page_soup = soup(page_html, 'html.parser')

    blog_card_headers = page_soup.find_all('div', class_='blog_card_header')
    
    # Extract href from a tags within these divs
    for header in blog_card_headers:
        a_tag = header.find('a')
        if a_tag and a_tag.get('href'):
            href = a_tag['href']
            # Optionally prepend the base URL if needed
            if not href.startswith('http'):
                href = 'https://www.innovestx.co.th' + href
            print(href)
            manual_links.add(href)

manual_links = list(manual_links)


https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-username
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-email
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-unlock
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-overall
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-connect
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-thaistock
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-tfex
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-notification
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-indicator
https://www.innovestx.co.th/products/product-user-guide/detail/manual/forgot-password-streaming-app
https://www.innovestx.co.th/products/product-user-guide/detail/manual/DA-Open-API-Manual
https://www.innovestx.co

## Extract Information and Save in Separate File

In [93]:
manual_links

['https://www.innovestx.co.th/products/product-user-guide/detail/manual/DA-manual-01',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/Manual-TFEX',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/topup-qrcode',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/DA-manual-webtrade08',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/Real-time',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/buy-mutal-fund',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/buy-fwd-insurance-with-invx-app',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-email',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/2-factor-authentication',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/buy-bond',
 'https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-username',
 'ht

In [94]:
for source_link in manual_links:
    response = requests.get(source_link,headers=headers)
    page_html = response.content
    page_soup = soup(page_html, 'html.parser')
    #useful content is in section_2
    s2 = page_soup.find('section', class_='section_2')

    text_content = []
    links = []
    image_urls = []

    # Extract text content
    for i,elem in enumerate(s2.find_all(['p','h1', 'h2', 'h3', 'div','li','span'])):
        if elem.get_text(strip=True, separator=' ') == '':
            pass
        elif elem.name in ['h1', 'h2','h3','li']:
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))
        elif elem.name == 'span' and 'class' in elem.attrs and ('font_main' in elem['class'] or 'font_light' in elem['class'] or 'font_medium' in elem['class']):
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))
        elif 'class' in elem.attrs and 'slide_description' in elem['class']:
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))
        elif 'class' in elem.attrs and 'blog_details__body_left_article_bullet_number' in elem['class']:
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))
        elif 'style' in elem.attrs and 'font-size: 15px' in elem['style']:
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))

    #Extract all links
    for a_tag in s2.find_all('a'):
        link = a_tag.get('href')
        if link:
            if not link.startswith('http'):
                link = 'https://www.innovestx.co.th' + link
            links.append(link)

    # Extract all image URLs
    for img_tag in s2.find_all('img'):
        img_url = img_tag.get('src')
        if img_url:
            if not img_url.startswith('http'):
                img_url = 'https://www.innovestx.co.th' + img_url.split('?')[0]
            image_urls.append(img_url)

    output_path = "/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/txt"
    filename = "innovestx_pug_"+source_link.split('/')[-1]+".txt"
    print(filename)
    with open(os.path.join(output_path,filename), 'w') as file:
        # Write text content
        file.write("Text Content:\n")
        for text in text_content:
            file.write(text + "\n")

        # Write links
        file.write("\nLinks:\n")
        for link in links:
            file.write(link + "\n")

        # Write image URLs
        file.write("\nImage URLs:\n")
        for img_url in image_urls:
            file.write(img_url + "\n")

# Print the extracted text, links, and image URLs
# print("Text Content:")
# for text in text_content:
#     print(text)

# print("\nLinks:")
# for link in links:
#     print(link)

# print("\nImage URLs:")
# for img_url in image_urls:
#     print(img_url)




innovestx_pug_DA-manual-01.txt
innovestx_pug_Manual-TFEX.txt
innovestx_pug_topup-qrcode.txt
innovestx_pug_DA-manual-webtrade08.txt
innovestx_pug_Real-time.txt
innovestx_pug_buy-mutal-fund.txt
innovestx_pug_buy-fwd-insurance-with-invx-app.txt
innovestx_pug_webtrade-email.txt
innovestx_pug_2-factor-authentication.txt
innovestx_pug_buy-bond.txt
innovestx_pug_webtrade-username.txt
innovestx_pug_dca-thai-stock-streaming-app.txt
innovestx_pug_DA-manual-webtrade07.txt
innovestx_pug_Register-InnovestX-App.txt
innovestx_pug_howtoresetpwandpinonapp.txt
innovestx_pug_sale-unit-trusts.txt
innovestx_pug_manual-invxpoint-offshore.txt
innovestx_pug_structured-note.txt
innovestx_pug_DA-manual-webtrade05.txt
innovestx_pug_webtrade-notification.txt
innovestx_pug_DA-manual-webtrade09.txt
innovestx_pug_webtrade-overall.txt
innovestx_pug_w8ben.txt
innovestx_pug_vdo-manual-streaming-app.txt
innovestx_pug_manual-invxpoint.txt
innovestx_pug_DA-manual-04.txt
innovestx_pug_ei-cashbalance-buy-fund.txt
innovestx_

## Create Dataset

In [95]:
bodies = []
titles = []
topics = []
subtopics = []
for source_link in manual_links:
    response = requests.get(source_link,headers=headers)
    page_html = response.content
    page_soup = soup(page_html, 'html.parser')
    #title
    h1_tag = page_soup.find('h1')
    #useful content is in section_2
    s2 = page_soup.find('section', class_='section_2')

    if h1_tag:
        title_text = h1_tag.get_text(strip=True)
        titles.append(title_text)
        print(title_text)
    else:
        titles.append('N/A')
        print('N/A')

    bodies.append(s2.get_text(strip=True, separator=' '))
    print(s2.get_text(strip=True, separator=' '))
    print('-----------------')
    topics.append('pug')
    subtopics.append(source_link.split('/')[-1])

df = pd.DataFrame(columns=['text','context','topic','subtopic'])
df['text'] = [title+"\n\n"+body for title, body in zip(titles,bodies)]
df['context'] = ['data/txt/innovestx_pug_'+subtopic+'.txt' for subtopic in subtopics]
df['topic'] = topics
df['subtopic'] = subtopics

Digital Asset : การถอนเงินบาท
การถอนเงินบาทจะมีอยู่ 2 ประเภทดังนี้ 1. ถอนเงินจากบัญชีสินทรัพย์ดิจิทัล ไปยังบัญชีบริหารเงินสด 2. ถอนเงินจากบัญชีบริหารเงินสด ไปยังบัญชีธนาคารไทยพาณิชย์ของลูกค้า 01 ถอนเงินจากบัญชีสินทรัพย์ดิจิทัล ไปยังบัญชีบริหารเงินสด 1. เลือก "สินทรัพย์ดิจิทัล" 2. เลือก "กระเป๋า" 3. เลือก "บัญชีสินทรัพย์ดิจิทัล" 4. เลือก "ถอนเงิน" 5. เลือกบัญชี 6. เลือกบัญชีบริหารเงินสด 7. ระบุจำนวนเงินที่ต้องการถอนไปยังบัญชีบริหารเงินสดโดยมีขั้นต่ำอยู่ที่ 2,000 บาท 8. ตรวจสอบข้อมูลและเลือก "ยืนยัน" 9. ใส่รหัส PIN 10. ได้รับรายการส่งคำสั่งสำเร็จ 02 การถอนเงินจากบัญชีบริหารเงินสด ไปยังบัญชีธนาคารไทยพาณิชย์ของลูกค้า 1. เลือก "บัญชีบริหารเงินสด" 2. เลือก "ถอนเงิน" 3. เลือกบัญชี 4. เลือกบัญชีที่ต้องการถอนเงิน 5. เลือกวิธีชำระเงินและระบุจำนวนเงินที่ต้องการถอน การเลือกวิธีการถอนเงินมี 2 วิธีดังนี้ 1. ถอนฟรี โดยเงินจะเข้าบัญชีของลูกค้าภายในเวลา 3 ชั่วโมงหลังจากทำรายการ 2. ถอนทันที โดยมีค่าธรรมเนียม 20 บาทต่อรายการ ขั้นต่ำในการถอนเงิน 2,000 บาท หมายเหตุ: ㆍรายการถอนเงิน ระหว่างเวลา 06.00 น. - 19

In [18]:
df = pd.read_pickle('/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/innovestx_pug.pkl')

In [19]:
df.iloc[0,0]

'Digital Asset : การถอนเงินบาท\n\nการถอนเงินบาทจะมีอยู่ 2 ประเภทดังนี้ 1. ถอนเงินจากบัญชีสินทรัพย์ดิจิทัล ไปยังบัญชีบริหารเงินสด 2. ถอนเงินจากบัญชีบริหารเงินสด ไปยังบัญชีธนาคารไทยพาณิชย์ของลูกค้า 01 ถอนเงินจากบัญชีสินทรัพย์ดิจิทัล ไปยังบัญชีบริหารเงินสด 1. เลือก "สินทรัพย์ดิจิทัล" 2. เลือก "กระเป๋า" 3. เลือก "บัญชีสินทรัพย์ดิจิทัล" 4. เลือก "ถอนเงิน" 5. เลือกบัญชี 6. เลือกบัญชีบริหารเงินสด 7. ระบุจำนวนเงินที่ต้องการถอนไปยังบัญชีบริหารเงินสดโดยมีขั้นต่ำอยู่ที่ 2,000 บาท 8. ตรวจสอบข้อมูลและเลือก "ยืนยัน" 9. ใส่รหัส PIN 10. ได้รับรายการส่งคำสั่งสำเร็จ 02 การถอนเงินจากบัญชีบริหารเงินสด ไปยังบัญชีธนาคารไทยพาณิชย์ของลูกค้า 1. เลือก "บัญชีบริหารเงินสด" 2. เลือก "ถอนเงิน" 3. เลือกบัญชี 4. เลือกบัญชีที่ต้องการถอนเงิน 5. เลือกวิธีชำระเงินและระบุจำนวนเงินที่ต้องการถอน การเลือกวิธีการถอนเงินมี 2 วิธีดังนี้ 1. ถอนฟรี โดยเงินจะเข้าบัญชีของลูกค้าภายในเวลา 3 ชั่วโมงหลังจากทำรายการ 2. ถอนทันที โดยมีค่าธรรมเนียม 20 บาทต่อรายการ ขั้นต่ำในการถอนเงิน 2,000 บาท หมายเหตุ: ㆍรายการถอนเงิน ระหว่างเวลา 06.00 น. 

In [97]:
df.to_pickle('/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/innovestx_pug.pkl')

## Extract Image

In [103]:
for source_link in manual_links:
    response = requests.get(source_link,headers=headers)
    page_html = response.content
    page_soup = soup(page_html, 'html.parser')
    #useful content is in section_2
    s2 = page_soup.find('section', class_='section_2')

    image_urls = []
    image_dir = '/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img'
    os.makedirs(image_dir, exist_ok=True)


    for img_tag in s2.find_all('img'):
        img_url = img_tag.get('src')
        if img_url:
            if not img_url.startswith('http'):
                img_url = 'https://www.innovestx.co.th' + img_url
            image_urls.append(img_url)

            # Download and save the image
            try:
                img_response = requests.get(img_url, headers=headers)
                img_response.raise_for_status()  # Check if the request was successful

                # Check if the content is an image
                if 'image' in img_response.headers.get('Content-Type', ''):
                    filename = 'innovestx_pug_'+source_link.split('/')[-1]+'__'+img_url.split('/')[-1].split('?')[0]
                    img_name = os.path.join(image_dir, filename)
                    with open(img_name, 'wb') as img_file:
                        img_file.write(img_response.content)
                    print(f'Downloaded and saved: {img_name}')
                else:
                    print(f'URL is not an image: {img_url}')
            except requests.exceptions.RequestException as e:
                print(f'Failed to download {img_url}: {e}')

Downloaded and saved: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_DA-manual-01__
Downloaded and saved: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_DA-manual-01__manual_ep2_withdraw-02.png
Downloaded and saved: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_DA-manual-01__manual_ep2_withdraw-03.png
Downloaded and saved: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_DA-manual-01__manual_ep2_withdraw-04.png
Downloaded and saved: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_DA-manual-01__manual_ep2_withdraw-05.png
Downloaded and saved: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_DA-manual-01__manual_ep2_withdraw-06.png
Downloaded and saved: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_DA-manual-01__manual_ep2_withdraw-07.png
Downloaded and saved: /Users/bsaejia/Documents

### Delete invalid file type

In [105]:
# Get a list of all files in the folder
all_files = glob.glob(os.path.join(image_dir, '*'))

# Define the allowed extensions
allowed_extensions = ('.png', '.jpg', '.jpeg')

# Iterate over all files and delete the ones that do not have the allowed extensions
for file_path in all_files:
    if not file_path.lower().endswith(allowed_extensions):
        try:
            os.remove(file_path)
            print(f'Deleted: {file_path}')
        except Exception as e:
            print(f'Failed to delete {file_path}: {e}')

print("File cleanup complete.")

Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_switch-mutual-funds__
Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_topup-scb-account__32.Picture52-copy
Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_mutual-fund-cancellation__
Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_structured-note__05.Pin_Default_Filling
Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_DA-manual-01__
Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_request-monthly-statement__
Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_topup-qrcode__32.Picture52-copy
Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_foreign-stock-trading__
Deleted: /Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img/innovestx_pug_s

## Generate Image Caption

In [3]:
image_dir = '/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/img'

print("number of image:", len(os.listdir(image_dir)))

number of image: 465


In [11]:
import sys

sys.path.append(os.path.abspath('../utils'))
from get_image_description import get_description
images = os.listdir(image_dir)
images_filepath = [os.path.join(image_dir, image) for image in images]
df = pd.DataFrame(columns=["text","content","topic","subtopic"])

df["content"] = images_filepath
df["topic"]   = "pug"
df["subtopic"] = [image.split("__")[0] for image in images]
df

,text,content,topic,subtopic
0,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_innovestx-refer-friends
1,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_sips-ats-account
2,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_sips-ats-account
3,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_Manual-FX
4,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_Intelligent-Portfolios
...,...,...,...,...
460,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_sips-ats-account
461,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_DA-manual-webtrade06
462,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_Notification-Tools
463,NaN,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_Intelligent-Portfolios


In [12]:
df["text"] = df["content"].apply(get_description)
df

,text,content,topic,subtopic
0,"The image is of a digital interface, likely a ...",/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_innovestx-refer-friends
1,The image appears to display a form in Thai la...,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_sips-ats-account
2,The image depicts a smartphone screen displayi...,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_sips-ats-account
3,The image is a screenshot of a mobile applicat...,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_Manual-FX
4,This image appears to show an investment portf...,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_Intelligent-Portfolios
...,...,...,...,...
460,"The image depicts an ""Amendment Form"" from ""Fu...",/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_sips-ats-account
461,The image is a screenshot of a cryptocurrency ...,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_DA-manual-webtrade06
462,The image is a screenshot of an iPhone lock sc...,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_Notification-Tools
463,The image shows a section of a mobile applicat...,/Users/bsaejia/Documents/2024_innovestx/innove...,pug,innovestx_pug_Intelligent-Portfolios


In [14]:
df.to_pickle('/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/innovestx_pug_img.pkl')